# Исследование надежности заемщиков

## Введение <a id="introduction"></a> 

### Бизнес-задача <a id="task"></a> 

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение, количество детей, уровень дохода и цель кредита клиента на факт погашения кредита в срок.

### План работы <a id="schedule"></a> 

1. [Введение](#introduction)
    - [Бизнес-задача](#task)
    - [План работы](#schedule)
2. [Загрузка данных](#data_load)
    - [Импорт необходимых библиотек](#library_import)
    - [Загрузка данных](#data_loading)
    - [Обзор и описание данных](#data_description)
3. [Предобработка данных](#preprocessing)
    - [Удаление пропусков](#gaps)
    - [Обработка аномальных значений](#abnormal_values)
    - [Удаление пропусков (продолжение)](#gaps_continue)
    - [Изменение типов данных](#data_types)
    - [Обработка дубликатов](#duplicates)
    - [Категоризация данных](#data_categorization)
4. [Исследование данных и ответы на вопросы](#data_research)
    - [Есть ли зависимость между количеством детей и возвратом кредита в срок?](#question_1)
    - [Есть ли зависимость между семейным положением и возвратом кредита в срок?](#question_2)
    - [Есть ли зависимость между уровнем дохода и возвратом кредита в срок?](#question_3)
    - [Как разные цели кредита влияют на его возврат в срок?](#question_4)
    - [Возможные причины появления пропусков в исходных данных.](#question_5)
    - [Почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.](#question_6)
5. [Общий вывод](#conclusion)

## Загрузка данных <a id="data_load"></a>

### Импорт необходимых библиотек <a id="library_import"></a> 

Импортируем библиотеку pandas.

In [39]:
import pandas as pd

### Загрузка данных <a id="data_loading"></a> 

Считаем данные из csv-файла в датафрейм и сохраним в переменную data.

In [40]:
try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv') # чтение файла с данными и сохранение в df

### Обзор и описание данных <a id="data_description"></a> 

Выведем первые 20 строчек датафрейма data на экран.

In [41]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Выведем основную информацию о датафрейме с помощью метода info().

In [42]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных <a id="preprocessing"></a> 

### Удаление пропусков <a id="gaps"></a> 

Выведем количество пропущенных значений для каждого столбца.

In [43]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. Один из них — days_employed. Другой столбец с пропущенными значениями — total_income — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца income_type. Например, у человека с типом занятости сотрудник пропуск в столбце total_income должен быть заполнен медианным доходом среди всех записей с тем же типом.

In [44]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений <a id="abnormal_values"></a>

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце days_employed. Обработаем значения в этом столбце: замените все отрицательные значения положительными с помощью метода abs().

In [45]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведем медианное значение трудового стажа days_employed в днях.

In [46]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставляем их как есть.

Выведем перечень уникальных значений столбца children.

In [47]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

В столбце children есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма data.

In [48]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

Ещё раз выведем перечень уникальных значений столбца children, чтобы убедиться, что артефакты удалены.

In [49]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Удаление пропусков (продолжение) <a id="gaps_continue"></a> 

Заполним пропуски в столбце days_employed медианными значениями по каждого типа занятости income_type.

In [50]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Убедимся, что все пропуски заполнены. Ещё раз выведем количество пропущенных значений для каждого столбца.

In [51]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных <a id="data_types"></a> 

Заменим вещественный тип данных в столбце total_income на целочисленный с помощью метода astype().

In [52]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов <a id="duplicates"></a> 

Обработаем неявные дубликаты в столбце education. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру.

In [53]:
data['education'] = data['education'].str.lower()

Выведем на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалим их.

In [54]:
data.duplicated().sum()

71

In [55]:
data = data.drop_duplicates()

### Категоризация данных <a id="data_categorization"></a> 

На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`.

In [56]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [57]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Выведем на экран перечень уникальных целей взятия кредита из столбца purpose.

In [58]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.

In [59]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [60]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## Исследование данных и ответы на вопросы <a id="data_research"></a> 

### Есть ли зависимость между количеством детей и возвратом кредита в срок? <a id="question_1"></a> 

Составим сводную таблицу, где индекс это будет количество детей, а значениями количество должников, количество кредиторов и доля должников.

In [61]:
data.pivot_table(index = 'children', values = 'debt', aggfunc = ['sum', 'count', 'mean']).reset_index().droplevel(1, axis=1)

,children,sum,count,mean
0,0,1063,14091,0.075438
1,1,444,4808,0.092346
2,2,194,2052,0.094542
3,3,27,330,0.081818
4,4,4,41,0.097561
5,5,0,9,0.000000


**Вывод:**

- люди, которые не имеют детей, лучше всего отдают кредит, доля должников здесь составляет с 7,5%;

- хуже обстоит дело с кредиторами, кто имеет одного или двух детей, здесь доля невозврата составляет соответственно 9,2% и 9,4%;

- остальные категории некорректно сравнивать, так как в них очень маленькая выборка.

### Есть ли зависимость между семейным положением и возвратом кредита в срок? <a id="question_2"></a> 

Составим сводную таблицу, где индексом будет семейный статус, а значениями количество должников, количество кредиторов и доля должников.

In [62]:
data.pivot_table(index = 'family_status', values = 'debt', aggfunc = ['sum', 'count', 'mean']).reset_index().droplevel(1, axis=1)

,family_status,sum,count,mean
0,Не женат / не замужем,273,2796,0.097639
1,в разводе,84,1189,0.070648
2,вдовец / вдова,63,951,0.066246
3,гражданский брак,385,4134,0.093130
4,женат / замужем,927,12261,0.075606


**Вывод:**

- У группы заемщиков с семейным статусом "женат / замужем" наблюдается наименьший процент задолжностей - 7.5%;
- В группах "Не женат / не замужем" и "гражданский брак" доля невозврата выше и составляет 9.7% и 9.3% соответственно;
- остальные категории сравнивать некорректно из-за недостаточной выборки.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок? <a id="question_3"></a> 

Составим сводную таблицу, где инлексами будут категории заемщиков, а значениями количество должников, количество кредиторов и доля должников.

Уточним категории заемщиков по уровню дохода:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [63]:
data.pivot_table(index = 'total_income_category', values = 'debt', aggfunc = ['sum', 'count', 'mean']).reset_index().droplevel(1, axis=1)

,total_income_category,sum,count,mean
0,A,2,25,0.080000
1,B,354,5014,0.070602
2,C,1353,15921,0.084982
3,D,21,349,0.060172
4,E,2,22,0.090909


**Вывод:**
- Чаще всего берут кредит люди в группе "С" с ежемесячным доходом от 50001 до 200000 рублей;
- Группы "A", "D" и "E" сравнивать некорректно из-за недостаточной выборки;
- В группе "B" задолженность встречается реже и составляет 7%;
- В группе "С" дела обстоят хуже, процент задолженности составляет 8,4%.

### Как разные цели кредита влияют на его возврат в срок? <a id="question_4"></a> 

Составим сводную таблицу, где индексами будут цели кредита, а значениями количество должников, количество кредиторов и доля должников.

In [64]:
data.pivot_table(index = 'purpose_category', values = 'debt', aggfunc = ['sum', 'count', 'mean']).reset_index().droplevel(1, axis=1)

,purpose_category,sum,count,mean
0,операции с автомобилем,400,4279,0.093480
1,операции с недвижимостью,780,10751,0.072551
2,получение образования,369,3988,0.092528
3,проведение свадьбы,183,2313,0.079118


**Выводы:**

- Чаще всего заемы совершают для операций с недвижимостью;
- Чаще всего задолженность встречается в группе операций с автомобилем и составляет 9.3%;
- Самые надежные заемщики совершают операции с недвижимостью, процент задолженностей в этой группе составляет 7.2%.

Вероятнее всего так происходит потому что финансовый порог входа в группу операций с недвижимостью достаточно высокий, благодаря часто обязательному первичному взносу и внушительной общей сумме кредита. Ипотеку берут люди в целом более состоятельные и часто состоящие в браке. Кроме того банки предъявляют самые жесткие требования к ипотечникам и тщательно их проверяют при выдаче кредитов.

### Возможные причины появления пропусков в исходных данных. <a id="question_5"></a> 

Пропуски в данных присутствуют в столбцах 'days_employed' и 'total_income'. Причем их количество и индексы строк совпадают. Пропуски не связаны с другими столбцами: заемщики с пропусками в данных разного возраста, имеют разные образование и семейный статус, источник дохода  цель займа.
Возможными причинами появления пропусков могут быть:
- заемщики брали кредит без подтверждения дохода;
- техническая ошибка при формировании таблицы с исходными данными (сбой во время выгрузки данных на анализ, либо при объединении данных в единый файл, который также должен был передаваться на анализ);
- эти данные не собирались.

### Почему заполнить пропуски медианным значением — лучшее решение для количественных переменных. <a id="question_6"></a> 

Некоторые значения могут сильно выделяться среди большинства. По этой причине в таких случаях лучше заполнять пропуски медианным значением, а не средним.

## Общий вывод <a id="conclusion"></a> 

1. **Дети**
- Люди, которые не имеют детей, лучше всего отдают кредит, доля должников здесь составляет с 7,5%;
- Хуже обстоит дело с кредиторами, кто имеет одного или двух детей, здесь доля невозврата составляет соответственно 9,2% и 9,4%;
- На погашение кредита в срок влияет не количество детей, а скорее их наличие.

2. **Семейное положение**
- У группы заемщиков с семейным статусом "женат / замужем" наблюдается наименьший процент задолжностей - 7.5%;
- В группах "Не женат / не замужем" и "гражданский брак" доля невозврата выше и составляет 9.7% и 9.3% соответственно;
- Семейное положение влияет на вероятную задолженность.

3. **Доход**
- Группы "A" (свыше 1 млн рублей), "D" (от 30 тыс. до 50 тыс. рублей) и "E" (до 30 тыс. рублей) сравнивать некорректно из-за недостаточной выборки;
- В группе "B" с доходом от 200 тыс. до 1 млн. рублей задолженность встречается реже и составляет 7%;
- В группе "С" с доходом от 50 тыс. до 200 тыс. рублей дела обстоят хуже, процент задолженности составляет 8,4%;
- Доход влияет на возврат кредита в срок.

4. **Цели кредита**
- Чаще всего задолженность встречается в группе операций с автомобилем и составляет 9.3%;
- Самые надежные заемщики совершают операции с недвижимостью, процент задолженностей в этой группе составляет 7.2%.
- Цель кредита влияет на его погашение в срок.

**Итого, самые надежные группы заемщиков**:
- Наличие детей: без детей;
- Семейное положение: женат / замужем;
- Доход: от 200 тыс. рублей в месяц;
- Цель кредита: операции с недвижимостью.